# Epic Seven Gear Simulator Testing Documentation
This notebook documents the testing process.

* The json data files are stored in the `/data` folder. The scripts to prepare these json file are also in the `/data` folder.
* The test modules (along with this notebook) are stored in the `/tests` folder.
* The class and function modules for the app are stored in the `/src` folder.
* `unittest` module is used for running the tests.

### Directory Structure Issue

Modules in `/tests` folder have to import data and modules from the `/data` or `/src` folders respectively, so we need to add the parent directory to the path.

This is solved by importing `set_directory` function from the `set_directory_function` module. As such, every testing module should have the following two lines of code at the top, so that the parent folder can be added to the path.

In [ ]:
from set_directory_function import set_directory
set_directory()

### Running the tests
The tests must be run from the root directory as:

python tests/test_module_name.py

e.g.,
python tests/test_get_stat_by_id.py

## Testing Validation Functions
In many of the functions that follow, the inputs need to be validated, so this section tests whether these validation functions work as intended.

### validate_stat_id()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Raises a ValueError input is None
* Raises a ValueError if input is not str. 
* Acceptable parameters: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 (as of now, if more stats are added this could increase later). Parameter can be entered as str or int.

Returns: str(stat_id)

Test Module: [test_validate_stat_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_stat_id.py)

Test Plan:
* stat_id = '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'. '10' (exactly as written in STATS file, should work)
* stat_id = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 (ints, should work)
* stat_id = -1 (some negative number, should be error)
* stat_id = 'Health' (some string that's not a number)
* stat_id = 111 (some large number as int that is not in acceptable parameter)
* stat_id = '111' (some large number as str that is not in acceptable parameter)
* stat_id = None (should raise ValueError)

Result: ALL PASSED.

### validate_gear_type()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Returns None if input is None
* Raises a ValueError if input is not str. 
* Raises a ValueError if stat_type and gear_type are provided and the stat_id isn't in the pool of allowed id's for that gear type and stat_type
* Acceptable parameters: 'Weapon', 'Helm', 'Armor', 'Necklace', 'Ring', 'Boots'. Parameters are converted to lower case before validating, so lower, upper or mixed case is acceptable.

Returns: gear_type.lower()

Test Module: [test_validate_gear_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_type.py)

Test Plan:
* gear_type = 'Weapon', 'Helm', 'Armor', 'Necklace', 'Ring', 'Boots' (exactly as written in TYPES file, should work)
* gear_type = 'weapon', 'helm', 'armor', 'necklace', 'ring', 'boots' (all lower case should still work)
* gear_type = 'weapOn', 'hElm', 'arMor', 'nEcklacE', 'rINg', 'BOOTS' (mix of lower and upper case, should work)
* gear_type = 'Sword', 'mask', 'MAIL', 'Jewelry', 'Accessory', 'Shoes' (random strings, should raise error)
* gear_type = 1 (non string parameter, should raise ValueError)
* gear_type = None (should return None)

Invalid Entries (check invalid combinations of gear_type with stat_id and stat_type):
* weapon, mainstat, 3
* armor, substat, 0
* helm, substat, 2
* necklace, mainstat, 8
* ring, mainstat, 10
* boots, mainstat, 6

Results: ALL PASSED.

### validate_stat_type()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates stat_type input and returns stat_type.lower()
* Defaults to 'mainstat' if input is None. mod default is False (used in parse_stat method)
* Raises a ValueError if input is not str. 
* Raises a ValueError if input is 'mainstat' and mod is True.
* Raises a ValueError if input is 'mainstat' and rolled is not equal to 0.
* Acceptable parameters: 'mainstat' and 'substat'. Parameters are converted to lower case before validating, so lower, upper or mixed case is acceptable.

Returns: stat_type.lower()

Test Module: [test_validate_stat_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_stat_type.py)

Test Plan:
* valid inputs (mod default False)
    * stat_type = 'mainstat', 'substat', 'Mainstat', 'Substat', 'MainSTAT', 'SUBstAt'
* invalid inputs (mod default False)
    * stat_type =  'mainstats', 'substats', 'subs', 'main', 'stats', 
    * stat_type = 1
    * stat_type = None
* valid inputs (mod True)
    * stat_type = substat', 'Substat','SUBstAt'
* invalid inputs (mod True)
    * stat_type = 'mainstat', Mainstat', 'MainSTAT'
    
* valid inputs 
    * rolled = 0 and 'mainstat' PASSED
    * rolled = 2 and 'substat' PASSED

* invalid inputs
    * rolled = 3 and 'mainstat' PASSED

Results: ALL PASSED.

### is_valid_stat_entry()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Checks whether an entry is a valid entry from the stats.json file.
* Checks whether the entry is a dict, if it has the keys 'id', 'text', and 'key_stat'
* If all conditions fulfilled, returns True
* If any of the conditions not fulfilled, returns False
* Acceptable parameters: Anything

Returns: boolean

Test Module: [test_is_valid_stat_entry.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_is_valid_stat_entry.py)

Test Plan:
* valid entries
    * stat_entry = all entries in stats.json (check all) PASSED
* invalid entries (integer, str, list, )
    * stat_entry =  1, 'stats', [1, 2, "stats"], None. PASSED
    * stat_entry = some random dict PASSED
    * stat_entry = some random dict containing an id key, an id key and text key PASSED

Results: ALL PASSED

### validate_selected_stats()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Accepts a list which can be empty or contain only valid id's.
* Checks whether the entry None or empty list : return empty list []
* Checks whether the entry is a list, if not raises ValueError
* Checks whether all the entries in the list are valid stat id's or not (by checking against keys from stats.json)
* Acceptable parameters: List, valid stat id's inside the list, empty list or None (both return empty list)

Returns: empty list or list containing valid stat objects

Test Module: [test_validate_selected_stats.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_selected_stats.py)

Test Plan:
* valid entries
    * selected_stats = [0], [0, 1], multiple. PASSED
    * selected_stats = ['3'], ['4', '10'], multiple. PASSED
    * selected_stats =  ['2', 8], multiple. PASSED

* invalid entries (integer, str, list, )
    * selected_stats = [], None. PASSED
    * selected_stats =  1, 'stats', [1, 2, 'health'], [{}, {'id' : 1}] PASSED

Results: ALL PASSED

### validate_rolled()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates whether 'rolled' input is an int in the range(0,6).
* Check whether entry is None: returns 0 by default
* Checks whether entry is int and whether it's in the range (0,6)
* Checks whether entry is 'mainstat', if it is, rolled cannot be anything but 0.
* Acceptable parameters: int [0, 1, 2, 3, 4, 5]

Returns: int in range(0, 6)

Test Module: [test_validate_rolled.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_rolled.py)

Test Plan:
* valid entries
    * rolled = [0, 1, 2, 3, 4, 5, 6]
    * repeat above, specifying stat_type = 'substat'
* invalid entries (None, str, list, negative, large int)
    * repeat above, removing the 0 and specifying stat_type = 'mainstat' (mainstats cannot have non-zero rolls)
    * rolled = None
    * rolled =  ['one', [1, 2], -1, 1000]

Results: 

### validate_mod()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates the mod parameter. It should be set to True when we're parsing a stat that is modded
* Check whether entry is bool: returns ValueError if it isn't a bool
* Aacceptable parameters: bool

Returns: bool

Test Module: [test_validate_mod.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_mod.py)

Test Plan:
* valid entries
    * mod = [True, False]
    * stat_type = 'substat'
* invalid entries
    * mod = None, str, list, negative, large int, 0, 1 (since we're testing for bool, 0 and 1 won't work)
    * stat_type = 'mainstat' and mod = True

Results: ALL PASSED

### validate_mod_type()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates the mod_type parameter. Used for modification system.
* If None provided, default to 'greater'
* Raises ValueError if it is not a str and if it isn't 'greater' or 'lesser' (not case sensitive)
* Aacceptable parameters: (str) 'greater' or 'lesser'

Returns: (str) 'greater' (default) or 'lesser'

Test Module: [test_validate_mod_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_mod_type.py)

Test Plan:
* valid entries
    * mod_type = ['greater', 'Greater', 'GreaTER']
    * mod_type = ['lesser', 'Lesser', 'LeSSer']
    * mod_type = None (should return 'greater')

* invalid entries
    * mod = some str, list, negative int, large int 

Results: 


### validate_gear_grade_input()

Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: Validates whether the input for gear_grade is a valid str. Acceptable parameters are: ['normal', 'good', 'rare', 'heroic', 'epic']. This is used inside the validate_gear_grade() function

Test Module: [test_validate_gear_grade_input.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_grade_input.py)

Test Plan: 
* valid inputs: 
    * ['normal', 'good', 'rare', 'heroic', 'epic'] (lower case)
    * ['Normal', 'Good', 'Rare', 'Heroic', 'Epic'] (Capitalized)
    * ['NORMAL', 'GOOD', 'RARE', 'HEROIC', 'EPIC'] (upper)
    * ['norMAL', 'GooD', 'rARe', 'HeRoIc', 'ePIC'] (mixed)
* invalid inputs: 
    * ['some str', 1, -1, 0, None]

Results: ALL PASSED

### validate_gear_grade()

Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: Validates whether the input gear grade is valid. Acceptable parameters are: ['normal', 'good', 'rare', 'heroic', 'epic']. Also, if substats/mainstats are provided, ensures that the gear_grade is appropriate and fits the number of substats. If no args provided, gets a random_gear_grade.

Test Module: [test_validate_gear_grade.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_grade.py)

Test Plan: 
* valid inputs: 
    * ['normal', 'good', 'rare', 'heroic', 'epic'] (lower case)
    * ['Normal', 'Good', 'Rare', 'Heroic', 'Epic'] (Capitalized)
    * ['NORMAL', 'GOOD', 'RARE', 'HEROIC', 'EPIC'] (upper)
    * ['norMAL', 'GooD', 'rARe', 'HeRoIc', 'ePIC'] (mixed)
* invalid inputs: 
    * ['some str', 1, -1, 0]
    * Number of substats provided exceed what's allowed on given gear_grade (e.g. Heroic can't have 4 substats as start)
    * Duplicate subs
    * Too many subs
    * Mainstat is in substat
    
* none input: 
    * Generate 1000 gear_grades and check whether they are in ['normal', good', 'rare', 'heroic', 'epic']
    
* valid input with substats but no gear_grade provided:
    * We should get back an appropriate gear grade based on number of substats provided. e.g. if 4 substats are provided, the gear must be an epic.

Results: ALL PASSED

### validate_gear_level()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Validates whether the input gear level is a valid gear level. If none provided, default to 85. Accept int only and in the range(58, 101)

Test Module: [test_validate_gear_level.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_level.py)

Test Plan: 
* valid inputs: [58, 69, 71, 85, 88, 90, 100]
* invalid inputs: [56, 57, 101, 102, -1. '70', 'level two', [], {}]
* none input: None (should return 85)


Results: ALL PASSED

### validate_gear_set()

Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: Validates whether the input gear set is valid. Acceptable parameters are one of the entries from sets.json such as 'health', 'attack', 'hit', 'speec', etc.

Test Module: [test_validate_gear_set.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_set.py)

Test Plan: 
* valid inputs: 
    * ['health', 'lifesteal', 'attack', 'destruction', 'speed'] (lower case)
    * ['Hit', 'Critical', 'Torrent', 'Immunity', 'Rage'] (Capitalized)
    * ['COUNTER', 'PROTECTION', 'REVENGE', 'PENETRATION', 'SPEED'] (upper)
    * ['countER', 'proTEction', 'REVenge', 'PenETRAtioN', 'spEEd'] (mixed)
* invalid inputs: 
    * ['some str', 1, -1, 0]
* none input: 
    * Generate 1000 gear_grades and check whether they are in list(SETS.keys())

Results: ALL PASSED

### validate_substat_ids()

Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: Validates whether the list of substat id's added to a gear manually are valid or not. Accepts only integers or a list of integers. The integers must be valid stat id's - range(0,11). Raises ValueError if more than 4 elements are provided, invalid stat_id's are provided, or if duplicate elements are provided. Also checks if mainstat id and substat ids are same.

Added new functionality to check against gear_type. If  gear_type is provided, it checks against the available pool to make sure there is no conflict.

Test Module: [test_validate_substat_ids.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_substat_ids.py)

Test Plan: 
* valid inputs: 
    * [1, 10, [1, 2, 3, 4], [4, 5, 6], [3, 4], [7]]
    * Same as above with valid mainstat ids
    * Same as above but provide valid gear types
    
* invalid inputs: 
    * [-1, 1000, '1', 'some str', [1, 2, 3, 4, 5], [1, 1], [1, 1, 3, 4], [2, 3, 4, 2], [3, 3, 3]]
    * valid_substat_ids [1, 10, [1, 2, 3, 4], [4, 5, 6], [3, 4], [7]] with invalid mainstat_ids
    * If gear_type provided and mainstat isn't in the pool of provided gear_type
    * If gear_type is provided and substat isn't in the pool of provided gear_type

Results: ALL PASSED

### validate_mainstat_id()

Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: Validates the mainstat id. Raises ValueError if mainstat id is not a valid stat_id or if the mainstat is also present in the substats.

Added new functionality to check against gear_type. If gear_type is provided, it checks against the available pool to make sure there is no conflict.

Test Module: [test_validate_mainstat_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_mainstat_id.py)

Test Plan: 
* valid inputs (substat_id=None): 
    * 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
    * '0', '1', '2', '3', '4',' 5', '6','7', '8', '9', '10'
    * Same as above with valid mainstat ids
    * Same as above but also include valid gear_type
    
* invalid inputs: 
    * [-1, 1000, 'some str', [1, 2]
    * valid mainstat id with subtat_id's 
    * valid_substat_ids with invalid mainstat_ids
    * valid substat and mainstat id's but invalid gear_type

Results: ALL PASSED

## Testing Utility Functions

### get_random_grade()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets a random gear grade - 'rare', 'heroic' or 'epic'. The rates of choosing them are 0.35, 0.53, and 0.12 respectively (rates taken from inside game and can be modified later in (prep_data_GRADE.py)[https://github.com/mesaqlain/e7_items/blob/main/data/prep_data_GRADE.py] file. No arguments needed.

Test Module: [test_get_random_grade.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_grade.py)

Test Plan: 
* Test whether the result we get is in ['normal', good', 'rare', 'heroic', 'epic']
* Generate 100,000 grades and test whether the results are as expected (as in if we get rare 35% of the time, heroic 53% of the time, epic 12% of the time, and the other two 0% of the time).

Test Implementation:
* Initialize starting variables and set value to 0.
* Use get_random_grade to pick a grade and increment the corresponding grade variable value by 1
* Repeat above 100,000 times.
* Get simulated rates
* Use assertAlmostEqual with delta value (tolerance level) of 0.02 to check whether the difference between actual and simulated rates is between 0.02.

Results: ALL PASSED

### get_gear_tier():

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets the tier of the gear based on the provided gear level. 
* Lvl 58-71: Tier 5
* Lvl 72-85: Tier 6
* Lvl 86-100: Tier 7

Returns: int 5, 6 or 7.

Test Module: [test_get_gear_tier.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_tier.py)

Test Plan: 
* valid inputs: 
    * 58-71: should return 5
    * 72-85: should return 6
    * 86-100: should return 7
    * None: should return 6
* invalid inputs: [56, 57, 101, 102, -1. '70', 'level two', [], {}] (should raise Value Error)

Results: ALL PASSED

### get_reforge_increase():

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: 
* Summary: Get the value by which a stat will increase by when an item has been reforged.
* Validates inputs
* Gets an int value from corresponding 'mainstat' or 'substat' section and the index based on 'rolled' count.
* NOTE: Moved to utilities.py file from stats.py

Returns: (int) STATS[stat_id]['reforge'][stat_type][rolled]

Test Module: 
* [test_get_reforge_increase.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_reforge_increase.py)
* [test_get_reforge_increase_in_class.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_reforge_increase_in_class.py) (Not currently relevant as it has been moved)

Test Plan:

* Valid entries ALL PASSED
    * stat_id = 0, None, None (if stat_type is None then default is 'mainstat', if rolled is None, then default is 0). Expected = 525
    * stat_id = 0, 'mainstat', 0. Expected = 525
    * stat_id = 0, 'substat'. Expected = 11
    * stat_id = 5, 'substat', 3. Expected = 5
    * stat_id = '5', 'substat', 3. Expected = 5


* Invalid entries ALL PASSED
    * stat_id = -1, None, None
    * stat_id = 0, 'some string', 2
    * stat_id = 3, 'substat', -1
    * stat_id = 4, 'mainstat', '1'
    
* Test after integrating into class (same as above tests): ALL PASSED
    * def setUp(self) and tearDown(self)
    * Assign self.stat.stat_id and self.stat.stat_type values before testing the function        

Results: ALL PASSED

### get_random_set()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets a random gear set from sets.json file (preparation of data is in prep_data_SETS.py)[https://github.com/mesaqlain/e7_items/blob/main/data/prep_data_SETS.py] file. No arguments needed.

Test Module: [test_get_random_set.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_set.py)

Test Plan: 
* Test whether the result we get is in sets.json
* Generate 100,000 sets and test whether each of the sets have been picked at least once

Test Implementation:
* Use get_random_set to pick a set 
* Add it to our set object of results
* Repeat above 100,000 times.
* Check whether we've picked every set at least once
* Use assertFalse to confirm

Results: ALL PASSED

### convert_int_to_str()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Takes all valid entries in a list and converts them to str. If only one int is provided, it converts it to str and returns it. If only one str is provided, it checks whether it is a number representation.

Returns: list containing str

Test Module: [test_convert_int_to_string.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_convert_int_to_string.py)

Test Plan: 
* Valid inputs:
    * [1, 1000, -1, [1, 2, 3], [1], '1', ['1'], ['10', '20'], ['3', 4]
    
* Invalid inputs:
    * 'some str', 'one'

Results: ALL PASSED

### get_random_gear_type()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets a random gear type from types.json file (preparation of data is in prep_data_TYPES.py)[https://github.com/mesaqlain/e7_items/blob/main/data/prep_data_TYPES.py] file. No arguments needed.

Test Module: [test_get_random_gear_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_gear_type.py)

Test Plan: 
* Test whether the result we get is in types.json
* Generate 100,000 sets and test whether each of the types have been picked at least once

Test Implementation:
* Use get_random_gear_type to pick a gear type
* Add it to our set object of results
* Repeat above 100,000 times.
* Check whether we've picked every gear type at least once
* Use assertFalse to confirm

Results: ALL PASSED

### get_gear_type_from_subs()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Retrieves a gear type based on given substat_id's restriction

Test Module: [test_get_gear_type_from_subs.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_type_from_subs.py)

Test Plan: 
* Check the default case (since no args provided, we'll get a random gear_type)
* Set gear_type='weapon' and no substat_id's, we expect to always get 'weapon'
* Set gear_type='ring' and no substat_id's, we expect to always get 'ring'
* Set gear_type='ring' and substat_ids=[], we expect to always get 'ring'
* gear_type=None, substat_ids=[0,2,4], with these cominations, we'll get ['ring', 'necklace', 'boots']
* gear_type=None, substat_ids=[0], with these cominations, we'll get ['ring', 'necklace', 'boots', 'helm']
* gear_type='weapon', substat_ids=[0], with these cominations, we'll get ['ring', 'necklace', 'boots', 'helm'] (even though we started with weapon)
* gear_type='ring', substat_ids=[10, 6, 7, 3], with these cominations, we'll always get ring, because these subs are valid on ring
* gear_type='weapon', substat_ids=[0, 3, 10, 8], with these cominations,  we'll get ['ring', 'necklace', 'boots', 'helm']

Test Implementation:
* Gnerate 100,000 sets and test whether each of the types have been picked at least once

Results: ALL PASSED

### get_mainstat_id()

Source: 
Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
Test: [test_functions.py]((https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py))

Function: Gets a random mainstat from the available pool of stats given gear_type. If substats are provided, ensures that the chosen mainstat does not collide with the substats.

Test Module: [test_get_mainstat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_type_from_subs.py)

Test Plan Invalid Entries: 
* No gear_type provided (check with main and sub not provided, just main provided, just sub provided, both sub and main provided)
* Invalid gear type provided (like some str)
* Invalid mainstats provided along with a gear type (such as weapon not having 2 as mainstat)
* Invalid substats provided along with a gear type (such as weapon not having any defense substats 4 and 5)
* Duplicate substats
* Too many substast (>4 )

Valid entries (repeat 1000 times to make sure all choices are made):
* Gear type provided, no mains or subs provided, we should get appropriate mainstat(e.g. weapon will always return 0, helm 2, armor 4, and the other 3 can return any mainstat)
* Gear type provided, if a valid mainstat is provided, we'll just get that mainstat back
* valid gear type, mainstat, and substats are provided, we'll get that mainstat back
* Gear type provided, valid substats are provided. Mainstats are returned in such a way that they don't clash with substats.

Results: ALL PASSED

### get_substat_ids()

Source: 
Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
Test: [test_functions.py]((https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py))

Function: Gets n random substat id's based on provided gear_type, gear_grade, mainstats, and substats. gear_type, gear_grade and mainstat_id must be provided. substat_ids is optional.
n = len(starting_subs) - len(given subs). e.g. If 3 substats are provided, and gear_grade is epic, then one more substat is randomly picked from available pool of id's (restrictions apply such as subs allowed on gear, cannot be duplicates, cannot be same as main)

Test Module: [test_get_substat_ids.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_substat_ids.py)

Test Plan Invalid Entries: 
* No args provided, only mainstat or substat provided, valid mainstat and substats provided, some invalid gear_type is provided
* Invalid mainstats and gear_type provided, invalid substats and gear_type are provided
* Duplicate subs, too many subs, gear grade doesn't match number of subs provided

Valid entries (repeat 1000 times to make sure all choices are made):
* Test all possible combinations of grade, type, and mainstat (no subs provided). The expected substat should be in the available gear pool except for the mainstat being added
* Gear type provided, if a valid mainstat is provided, we'll just get that mainstat back
* valid gear type, mainstat, and substats are provided, we'll get that mainstat back
* Gear type provided, valid substats are provided. Mainstats are returned in such a way that they don't clash with substats.

Results: ALL PASSED

### get_random_stat_id()

Source: 
Main: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets a random stat_id from available pool of id's based on gear_type. If no gear_type provided, then any of the valid stat_ids from stats.json may be chosen

Returns: stat_id (str)

Test Module: [test_get_random_stat_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_stat_id.py)

Test Plan Valid Entries (repeat 1000 times to make sure all choices are made): 
* No args provided, so expected pool is all stat ids, and output should be str
* stat_type = 'mainstat', expected pool is all stat ids, and output should be str
* stat_type = 'substat', expected pool is all stat ids, and output should be str
* stat_type = 'mainstat', gear_type=try them all, (expected should be mainstat pool from that gear type) and output should be str
* stat_type = 'substat', gear_type=try them all, (expected should be substat pool from that gear type) and output should be str
* stat_type not specified (so default substats), gear_type=try them all, (expected should be substat pool from that gear type) and output should be str

Results: ALL PASSED

### get_non_overlapping_stat_id()

Source: 
Main: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets a random stat_id not already in the selected stats, given the gear_type . If no gear_type provided, then any of the valid stat_ids from stats.json that are not already in selected may be chosen.

Returns: stat_id (str)

Test Module: [test_get_non_overlapping_stat_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_non_overlapping_stat_id.py)

Test Plan Valid Entries (repeat 1000 times to make sure all choices are made): 
* Run all tests from get_random_stat_id() section by not specifying anything for selected_stats, and also selected_stats = None and []. The results should be exactly the same.
* Use one of the stats from stats.json as our selected stat without specifying gear_type (so we expect to see all stats other than the one in selected)
* Specify 2-4 selected stats without specifying gear_type (so we expect to see all stats other than the ones in selected)
* Specify selected stats as well as gear_type, this limits the substats we should expect.

Test Invalid Entries:
* Invalid stat entries such as large ints or negative numbers.
* Not thoroughly tested as the functions used within this function has been tested thoroughly before. 

Results: ALL PASSED

### check_valid_pool()

Source: 
Main: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Raises an error if the provided stat_ids are not in the allowed pool of stat_type for given gear(such as atk (0) not allowed as mainstat on weapon, or atk% (1) not allowed in substats on armor)

Returns: Nothing, but raises error if the ids are not in valid pool

Test Module: [test_check_valid_pool.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_check_valid_pool.py)

Test Plan InValid Entries (check if they raise error): 
* no args provided
* gear_type provided with incorrect mainstat (default is mainstat if no stat_type specified)
* different combination of gear_types with stat_types and provided stat_ids to check if they raise error

Test Valid Entries (had to write a function for notRaises):
* Valid mainstat ids for weapon, helm and armor
* Valid substat ids for different gear types

Results: ALL PASSED

## Testing Stat Class

### get_stat_by_id()

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: Retrieves data on a STAT based on the given id. It returns a dictionary.

Test Module: [test_get_stat_by_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat_by_id.py)

The Stat() class initiates with no data. Running the get_stat_by_id() method should return a dictionary from the STATS dictionary based on the id provided. The function should be able to handle both int and str arguments as long as the number is between 0 and 10. So we need to test the following cases.

Valid stat_id should return a dictionary containing stats where id is 0 or 7 (compare to STATS[0] and STATS[7]):
* stat_id = 0 
* stat_id = '0' 
* stat_id = 7 
* stat_id = '7' 
* stat_id = 0, but check whether the store selected_stat_id is correctly returned as 0.
* stat_id = '7' , but check whether the store selected_stat_id is correctly returned as 0.


Invalid stat_id should raise ValueError:
* stat_id = 100
* stat_id = '100'
* stat_id = -1 (negative case)
* stat_id = 1000000 (big number)
* stat_id = 'Health' (some random string)
* stat_id = None

Additional tests:
* Check whether we get the correct stat_id and stat_key attributes
* Check whether we correctly get stat_type='mainstat' and gear_type=None if none provided
* Check whether we correctly get self.stat_type and self.gear_type if these attributes are provided

Additional ValueErrors (invalid combinations of gear_type with stat_id and stat_type):
* weapon, mainstat, 3
* armor, substat, 0
* helm, substat, 2
* necklace, mainstat, 8
* ring, mainstat, 10
* boots, mainstat, 6

Result: ALL PASSED

### get_random_stat()
Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Retrieves data on a random STAT chosen from the list of STATS in the stats.json file. 
* Validates gear_type input
* Initializes an empty list called pool
* If gear_type is None, the pool will contain all stats from the stats.json file.
* If gear_type is provided, the pool will be limited to the stats allowed according to stat_type (for more information on allowed stats on specific gear type, refer to prep_data_TYPES.py module or check Section 5.2 in the README file.)
* Chooses a random id from the pool and runs the get_stat_by_id() function to get that stat.

Returns: Stat dic

Test Module: [test_get_random_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_stat.py)

Test Plan:
* get_random_stat() with no inputs (this means it should use 'mainstat' as default gear_type = None)
    * Check that we do get a dictionary that is one of the stats entries in stats.json PASSED 
    * Generate 1000 random stats and count how many times each stat was chosen, they should all have been chosen at least once. PASSED

* Valid stat_type inputs (while gear_type = None):
    * Repeat above test by specifying stat_type = 'mainstat'. PASSED
    * Repeat above test by specifying stat_type = 'substat'. PASSED

* Valid gear_type inputs (while stat_type = None):
    * Repeat above test by specifying gear_type = 'weapon' (we expect to see the stat_id = 0 only (because Weapons can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'helm' (we expect to see the stat_id = 2 only (because Helms can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'armor' (we expect to see the stat_id = 4 only (because Armor can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'necklace' (we expect to see no stat_id = 8,9,10 (because Necklace cannot have those as mainstat), while others should be chosen at least once)  PASSED
    * Repeat above test by specifying gear_type = 'ring' (we expect to see no stat_id = 6,7,10 (because Ring cannot have those as mainstat), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'boots' (we expect to see no stat_id = 6,7,8,9 (because Boots cannot have those as mainstat), while others should be chosen at least once) PASSED
    
* Valid gear_type inputs (while stat_type = 'mainstat'):
    * Repeat all tests in the previous section by specifying stat_type = 'mainstat', we should see same results ALL PASSED

* Valid gear_type inputs (while stat_type = 'substat'):
    * Repeat above test by specifying gear_type = 'weapon' (we expect to see no stat_id = 0, 4, 5 (because Weapons cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'helm' (we expect to see no stat_id = 2 (because Helms cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'armor' (we expect to see no stat_id = 0, 1, 4 (because Armors cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'necklace', 'ring', 'boots' (we expect see all stats chosen at least once) PASSED
    
* Invalid stat_type inputs (while gear_type = None):
    * stat_type = 111, '111', 'health' (test integers, random strings, etc) PASSED
    
* Invalid gear_type inputs (while stat_type = None):
    * gear_type = 111, '1', 'Sword' (test integers, random strings, etc) PASSED

* Invalid inputs for both:
    * gear_type = 111, stat_type = 'Sword' PASSED
    
* Additional Tests to check if correct selected_stat, stat_type, gear_type is returned
    * gear_type = 'ring', stat_type = 'mainstat' PASSED
    * gear_type = 'helm', stat_type = 'substat' PASSED

Test Plan Implementation:
* Create an empty set called selected_stat_ids = set()
* Generate 1000 random stats and take the 'id' of those stats and add it to selected_stat_ids
* Define expected_stat_ids, which contains the allowed stats for a particular gear type (based on stats)
* Calculate missing_stat_ids = expected_stat_ids - selected_stat_ids
* Calculate extra_stat_ids = selected_stat_ids - expected_stat_ids
* If missing_stat_ids set is empty, it means that all the expected stats were selected, and the assertion will pass.
* If extra_stat_ids set is empty, it means that no extra stats were selected, and the assertion will pass.

Further Test Implementation:
* As more Stat class attributes are added, if needed, they can be aded to the test.

Results: ALL PASSED

### get_non_overlapping_stat():

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Retrieves data on a random stat that is not already in the pool of 'selected stats'. 
* Validates inputs
* Gets a random stat
* If that random stat already exists in the selected pool, get another random stat.
* Repeat until a stat not already in the pool in selected.
* Return the new stat
* Chooses a random id from the pool and runs the get_stat_by_id() function to get that stat.

Returns: Stat dic

Test Module: 
* [test_get_non_overlapping_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_non_overlapping_stat.py)
* [test_get_non_overlapping_stat_in_class.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_non_overlapping_stat_in_class.py)

Test Plan:

* Use empty list and None for selected stats PASSED
    * Check that we do get a dictionary that is one of the stats entries in stats.json  
    * Generate 1000 random stats and count how many times each stat was chosen, they should all have been chosen at least once if empty list was provided. 

* Valid selected_stats inputs: PASSED
    * We'll create a list with each of the STATS entries and generate 1000 non-overlapping stats, none of those stats should contain the one in the seleceted_list.
    * A list containing two chosen stats from stats.json, and another containing three.
    * We'll repeat the above test, making sure that the returned output is not the same as one of the selected stats.

* Valid gear_type inputs (while stat_type = 'substat'): PASSED
    * Repeat above test by specifying gear_type = 'weapon' (we expect to never see stats 0, 4, 5 + the stat that was in selected stats 
    * We won't test for any other gear_type because this was tested extensively for get_random_id().
    
* We repeat some of the tests from above by incorporating the function inside the Stat class PASSED

* Additional Test after adding selected_stat_id class attribute
    * Run some of the above valid tests and check whether the stored selected_stat_id is correct.
    
Results: ALL PASSED

### get_mod_value()

Source: 
* Testing: [test_functions.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py)
* Actual: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: 
* Summary: Choose a value for a modified stat based on given rolled_count, gear_level, and mod_type
* Validates inputs
* Get the list of mod values for the given criteria and mod_type
* Checks gear_level and rolled_count to get the list of values
* Randomly pick a value from the list

Returns: (int) value of stat

Test Module: 
* [test_get_mod_value.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_mod_value.py)

Test Plan:

* Valid entries ALL PASSED
    * specify stat_id = 0 (attack_flat) (leaving others blank, so default lv=85, mod_type=greater, rolled=0). We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85 (others default). We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85, rolled=0. We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85, rolled=0, mod_type='greater'. We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85, rolled=0, mod_type='lesse'. We expect value between 28-40
    * specify stat_id = 7 (crit_damage) (other arguments default) (we expect values 4-7)
    * specify stat_id = 6 (crit_chance), rolled=1 (other args default) (expected 3-6)
    * specify stat_id = 5 (defense_percent), rolled=2, mod_type='lesser' (expect 8-11)
    * specify stat_id = 10 (speed), rolled=4, mod_type='greater', gear_level=90 (expect 10-13)
    * specify stat_id = 2 (flat_health), rolled=5, mod_type='lesser', gear_level=88 (expect 477-560)

* Invalid entries ALL PASSED
    * stat_id = 1, gear_level=101 (invalid gear level)
    * stat_id = -1, (invalid stat_id)
    * stat_id = 10, mod_type = 'big mod' (invalid mod_type)
    * stat_id = 5, rolled = 6
    * stat_id = 5, rolled = -1
    
* Test implementation
    * Perform each test 10,000 times to check that every value has been picked at least once.
    
* Test after integrating into class


Results: 

### get_stat_value()

Source: 
* Testing: [test_functions.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py)
* In class: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: 
* Summary: Choose a value for a non-modified stat based on stat_id, stat_type, gear_level, and gear_grade
* Validates inputs and gets gear tier
* Get the list of stat values for the given criteria 
* If fixed (main), assigns value; if rand (sub), chooses a value based on rates

Returns: (int) value of stat

Test Module: 
* [test_get_stat_value.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat_value.py)

Test Plan:

* Valid entries 
    * specify stat_id = 0 (attack_flat) (leaving others blank, so default lv=85, 'mainstat', grade=random). We expect 100 (same val for all grades) 
    * specify stat_id = 0 (attack_flat), lv=85 (others default). We expect value 100 
    * specify stat_id = 0 (attack_flat), lv=85, 'mainstat'. We expect value 100  
    * specify stat_id = 0 (attack_flat), lv=85, 'mainstat', grade='rare'. We expect value 100  
    * specify stat_id = 0 (attack_flat), lv=90, 'mainstat', grade='rare'. We expect value 103  
    * specify stat_id = 7 (crit_damage) (other arguments default) (we expect value 13)
    * specify stat_id = 6 (crit_chance), lv=88, 'substat', grade = 'epic (we expect values 3-6)
    * specify stat_id = 5 (defense_percent), lv=70, 'substat', 'grade'='rare' (we expect values 4-6)
    * specify stat_id = 10 (speed), 'substat', gear_level=90, grade='epic' (expect 3-5)

* Invalid entries 
    * No need to check as we've thoroughly tested the input validation functions before
    
* Test implementation
    * Perform each test 10,000 times to check that every value has been picked at least once.
    
* Test after integrating into class

Results: ALL PASSED

### parse_stat()

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Probably the most important method for the Stat class. It provides a fixed value for a stat based on its type ('substat' or 'mainstat') for given gear grade and level, and whether it is being modded or not.
* Validates inputs and gets gear_tier from gear_level
* Assigns the following class attributes:
    * self.text 
    * self.gear_grade
    * self.gear_level 
    * self.gear_tier 
    * self.rolled 
* Checks if modded is True or False (default is False) and gets the appropriate stat value based on this condition
* If modded, sets self.modded to True (default is False)
* Further assigns the following attributes:
    * self.value 
    * self.value_key
    * self.reforge_increase 
    
Returns self

Test Module:
* [test_parse_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_parse_stat.py)

Test Plan:
* Test mod=False PASSED
    * initiate get_random_id() (we're testing whether we get the correct class attributes for the following tests)
    * all None 
    * gear_grade='heroic' (testing whether we get the correct gear_grade)
    * gear_grade='epic', gear_level=88 (expect tier 7)
    * stat_type='substat', gear_grade='rare', gear_level=88, rolled=3 (expect tier 7)
    * stat_type='substat', gear_grade='epic', gear_level=70, (expect tier 5)
    * stat_type='substat', gear_grade='epic', gear_level=70, gear_type='boots' (expect tier 5)

* Invalid PASSED
    * stat_type='mainstat', gear_grade='heroic', gear_level=88, rolled=3 (can't have rolled non zero on mainstat)
    * gear_grade='heroic', gear_level=88, rolled=2 (can't have rolled non zero on 'mainstat' which is default)
    
* Test mod=False PASSED
    * initiate get_stat_by_id(6) (this always makes the stat crit_chance, repeat experiments 1000 times to see al values were chosen at least once)
    * all None (expect stat_id='6', stat_key='crit_chance', stat_type='mainstat', gear_type='None', grade=random,
        gear_level=85, gear_tier=6, rolled=0, value=11 (this should be always true regardless of grade),
        value_key='\<A>', reforge_increase=60)
    * stat_type='substat', gear_level='90', gear_grade='rare' (expect stat_id='6', stat_key='crit_chance', stat_type='substat', gear_type=None, grade='rare',
        gear_level=90, gear_tier=7, rolled=0, values=[3,4,5], value_key='\<A>', reforge_increase=1)
    * stat_type='substat', gear_level='70' gear_grade='epic' (expect stat_id='6', stat_key='crit_chance', stat_type='substat', gear_type=None, grade=epic,
        gear_level=70, gear_tier=5, rolled=0, values=[2,3,4], value_key='\<A>', reforge_increase=1)


* Test mod=True invalid cases PASSED
    * initiate get_stat_by_id(10) (this always makes the stat speed, repeat experiments 1000 times to see all values were chosen at least once)
    * all None (default is mainstat so we should get error) try get_stat_by_id(), get_random_stat(), get_non_overlapping_stat()
    
     
* Test mod=True, valid cases, speed (stat_id=10) PASSED
    * initiate get_stat_by_id(10) (this always makes the stat speed, repeat experiments 1000 times to see all values were chosen at least once)
    * stat_type='substat', mod=True, gear_level=90, gear_grade='epic', rolled=4
    * stat_type='substat', mod=True, gear_level=85, gear_grade='heroic', rolled=2, mod_type='lesser'


* Test mod=True, valid cases, get_non_overlapping PASSED
    * [0,1,2,3], stat_type='substat', gear_type='ring', mod=True, rolled=2, mod_type='lesser'
    * [6,7,8,9], stat_type='substat', gear_type='armor', gear_level=88, mod=True, rolled=5, mod_type='greater'

### format_stat()

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Returns a concise one line representation of the stat with its value and stat name.
* Gets the appropriate reforge value depending on whether stat is mainstat or substat 
    * For mainstat, reforged value is self.reforge_increase 
    * For substat, reforged value is self.reforge_increase + self.value
* Checks if the stat is modded, if it is, appropriately shows (modded)
    
Returns str

Test Module:
* [test_format_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_format_stat.py)

Test Plan: We'll check some very specific scenarios to see whether we get the expected results
* Mainstat: PASSED
    * Percent stat, no mod no reforge
    * Flat stat, no mod no reforge
    
* Substat: PASSED
    * Percent stat, no mod no reforge
    * Percent stat, modded, no reforge
    * Percent stat, modded, reforged
    * Percent stat, modded, reforged, rolled = 3
    * Flat stat, no mod no reforge
    * Flat stat, no mod, reforged
    * Flat stat, modded, no reforge
    * Flat stat, modded, reforged


### enahnce_stat()

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Enhances a stat value.
* Mainstat 
    * Depends on enhance_level of gear
    * Gets the based value of stat at +0 enhance and multiplies with a multiplier based on enhance_level
    * Updates stat value and text_formatted
* Substat 
    * Depends on rolled count
    * Gets a random value for the stat based on gear_level and gear_grade
    * Adds that value to current value to get enhanced value 
    * Increments rolled count by 1
    * Updates reforge increase and text_formatted
    
Returns self

Test Module:
* [test_enhance_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_enhance_stat.py)

Test Plan: We'll check some very specific scenarios to see whether we get the expected results
* Mainstat: PASSED
    * Test weapon, so mainstat is attack, base value should be 100 at enhancement level 0. Enhancing it should increase value to 120.
    * Test necklace, specify mainstat crit chance, fix value to 14 (this is expected at enhance level +1), and check if we get enhance level 17 at +2.
    * Test boots, specify mainstat speed, fix value to 34 (enhance level at +14), expect 40 at +15.
    * Test sequential increase for a stat from +0 to +15, choose Eff % for Ring. 
    
* Substat: PASSED
    * Test speed substat, check if rolled count is appropriately increased, as well as reforge_increase. Check if the updated value falls in the range we expect.
    * Test atk% substast, set rolled to 3. Check if the updated rolled count and reforge_increase reflects the enhancement. Check if value falles in expected range.
    
Results: ALL PASSED

## Testing Gear Class

### Gear()

Function: Gear class that holds mainstats and substats with enhance, reforge, and modify methods

Source: 
* Main: [gear.py]((https://github.com/mesaqlain/e7_items/blob/main/src/gear.py))


Test Modules:
* test_create_gear_v1


### create_gear() -v1

Function: Method to create a new gear. If arguments are set to None, a completely random gear of level 85 is created.

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test V1: [test_class_gear_v1.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_class_gear_v1.py)

Test Modules:
* Test V1: [test_create_gear_v1.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_create_gear_v1.py)

Test Plan V1:
* Create an empty Gear() instance and check if the initial values are as we expect
* Use .create_gear() method with default args and check if values are as we expect
* Incrementally add various args and check whether we get values as expected
* Check invalid cases such as duplicate substats or mainstat and substats being same


### get_gear_type()

Function: Retrieves a random gear type based on provided mainstat_id and substat_id. Used in the create_gear() method in Gear() class.

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test: [test_functions.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py)

Test Module:
* Test: [test_get_gear_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_type.py)
* In class: [test_get_gear_type_in_class.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_type_in_class.py)
 
Test Plan Valid Entries: ALL PASSED
* Check defaults (gear_type=None, mainstat_id=None, substat_ids=None) - expect to get every gear type PASSED
* provide mainstat_id only (we should get appropriate gear_type, for instance, if mainstat=10 (speed), our only choice of gear is boots)
* provide substat_ids only (we should get appropriate gear_type)
    * Check by providing only one stat
    * Check by providing multiple combinations of stats
* gear_type=['ring', 'weapon'. 'helm', 'armor', 'boots', 'necklace'], mainstat_id=None, substat_ids=None - expect to get the same as input since other args none
* Check if gear_type and mainstat match, we get the gear_type back
* Check if gear_type and substats match, we get the gear_type back
* Check if mainstats and substats are valid, we get the appropriate gear back
* If all args are valid, return gear as is

Test Plan Invalid Entries: ALL PASSED
* Incorrect gear_type and mainstats
* Incorrect gear_type and mainstats (even if substats are correct)
* Incorrect gear_type and substats (even if mainstat is correct)
* Incorrect gear_type and substats (with mainstat provided)
* Duplicate substats (gear_type and mainstat not provided)
* Duplicate substats (even if gear_type is correct)
* Duplicate substats (even if gear_type and mainstats are correct)
* Mainstat is used in substat (gear_type not provided)
* Mainstat is used in substat (even if gear_type is correct)
* Too many substats


### create_gear() - v2
**get_gear_type()** and **validate_gear_grade()** implemented

Function: Method to create a new gear. If arguments are set to None, a completely random gear of level 85 is created. Gear_Type is now set randomly if no args provided, given mainstat and substat restrictions. Gear_Grade is also validated based on number of substats provided. If non 

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test V2: [test_class_gear_v2.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_class_gear_v2.py)

Test Modules:
* [test_create_gear_v2.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_create_gear_v2.py)
 
Test Plan Valid Entries:
* Create an empty Gear() instance and check if the initial values are as we expect
* Use .create_gear() method with default args and check if values are as we expect
* Specify gear_type and check if we get same output for gear_type, and one of the random gear_grades
* Specify gear_type, gear_set and check if we get same output, as well as a random gear_grade
* Specify gear_type, gear_set, and gear_level, and see if we get appropriate outputs, as well as a random gear_grade
* Specify gear_type, gear_set, and gear_level, gear_grade, and see if we get appropriate outputs
* Specify mainstat, gear_set, and gear_level, gear_grade, and see if we get appropriate gear_type outputs
* Specify mainstat, substats, gear_set, gear_level, and see if we get appropriate gear_type outputs, as well as expected gear_grades
* 

Test Plan Invalid Entries:
* Duplicate substats 
* Mainstat and substats being same
* Incorrect mainstat and gear_type
* Incorrect substat and gear_type
* Incorrect number of substats with gear_grade

### create_gear() - v3
**get_gear_type()**, **validate_gear_grade()**, **get_mainstat_id()** implemented

Function: Method to create a new gear. If arguments are set to None, a completely random gear of level 85 is created. Gear_Type is now set randomly if no args provided, given mainstat and substat restrictions. Gear_Grade is also validated based on number of substats provided. If no mainstat_id provided, it'll get a random mainstat_id based on given substats and gear_type. if incorrect combination of mainstats/substats/gear_type are provided, error is raised. 

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test V3: [test_class_gear_v3.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_class_gear_v3.py)

Test Modules:
* [test_create_gear_v3.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_create_gear_v3.py)
 
Test Plan Valid Entries:
* Create an empty Gear() instance and check if the initial values are as we expect
* Use .create_gear() method with default args and check if values are as we expect
* Specify gear_type and check if we get same output for gear_type, one of the random gear_grades, and an appropriate mainstat_id
* Specify gear_type, gear_set and check if we get same output, as well as a random gear_grade and an appropriate mainstat_id
* Specify gear_type, gear_set, and gear_level, and see if we get appropriate outputs, as well as a random gear_grade and an appropriate mainstat_id
* Specify gear_type, gear_set, and gear_level, gear_grade, and see if we get appropriate outputs and an appropriate mainstat_id
* Specify mainstat, gear_set, and gear_level, gear_grade, and see if we get appropriate gear_type outputs
* Specify mainstat, substats, gear_set, gear_level, and see if we get appropriate gear_type outputs, as well as expected gear_grades
* Specify substats, gear_set, gear_level gear_type, and check if we get an appropriate main stat and a proper gear_grade

Test Plan Invalid Entries:
* Duplicate substats 
* Mainstat and substats being same
* Incorrect mainstat and gear_type
* Incorrect substat and gear_type
* Incorrect number of substats with gear_grade
* Too many substats >4

### create_gear() - v4
**get_gear_type()**, **validate_gear_grade()**, **get_mainstat_id()** implemented

Function: Method to create a new gear. If arguments are set to None, a completely random gear of level 85 is created. Gear_Type is now set randomly if no args provided, given mainstat and substat restrictions. Gear_Grade is also validated based on number of substats provided. If no mainstat_id provided, it'll get a random mainstat_id based on given substats and gear_type. If no substats are provided, it'll pick random substats based on gear and other restrictions. If incorrect combination of mainstats/substats/gear_type are provided, error is raised. 

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test V4: [test_class_gear_v4.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_class_gear_v4.py)

Test Modules:
* [test_create_gear_v4.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_create_gear_v4.py)
 
Test Plan Valid Entries:
* Create an empty Gear() instance and check if the initial values are as we expect
* Use .create_gear() method with default args and check if values are as we expect
* Specify gear set and level
* Specify set, level, type 
* Specify different sets of mainstats and substats and check whether we get appropriate grade and type.

Test Plan Invalid Entries:
* Duplicate substats 
* Mainstat and substats being same
* Incorrect mainstat and gear_type
* Incorrect substat and gear_type
* Incorrect number of substats with gear_grade
* Too many substats >4
* Incorrect gear_grade with no. of substats provided

Results: ALL PASSED

### get_stat()

Function: get_stat method for Gear() class to retrieve stat information based on given attributes. All attributes must be provided.

Returns: stat object that has been parsed already

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test: [test_functions.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py)

Test Modules:
* [test_get_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat.py)

Test Plan Valid Entries:
* Try 6 different cases with valid entries and check if we get expect parsed values

Test Plan Invalid Entries:
* Tweak the above 6 cases by making some of the attribtues invalid

Results: ALL PASSED

### create_gear() - v5
**get_gear_type()**, **validate_gear_grade()**, **get_mainstat_id()**, **get_stat()** method implemented.

Function: Method to create a new gear. If arguments are set to None, a completely random gear of level 85 is created. Gear_Type is now set randomly if no args provided, given mainstat and substat restrictions. Gear_Grade is also validated based on number of substats provided. If no mainstat_id provided, it'll get a random mainstat_id based on given substats and gear_type. If no substats are provided, it'll pick random substats based on gear and other restrictions. If incorrect combination of mainstats/substats/gear_type are provided, error is raised. In this version, after getting the stat_id's for mainstat and substat, the get_stat() method parses those stats and stores the values in self.mainstat and self.substats.

Source: 
* Main: [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)
* Test V5: [test_class_gear_v5.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_class_gear_v5.py)

Test Modules:
* [test_create_gear_v5.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_create_gear_v5.py)
 
Test Plan Valid Entries:
* Create an empty Gear() instance and check if the initial values are as we expect
* Use .create_gear() method with default args and check if values are as we expect
* Specify mainstat and substat ids and check if we retrieved and parsed appropriate values based on stat provided

Test Plan Invalid Entries:
* Duplicate substats 
* Mainstat and substats being same
* Incorrect mainstat and gear_type
* Incorrect substat and gear_type
* Incorrect number of substats with gear_grade
* Too many substats >4
* Incorrect gear_grade with no. of substats provided

Results: ALL PASSED

### get_gear_score()

Function: Method within the gear class that returns the standardized gear score, both before and after reforge

Source: 
* [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)

Test Modules:
* [test_get_gear_score.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_score.py)
 
Test Plan Valid Entries:
* Create an empty gear object and then create_gear(), specifying the gear_grade and substats (so we know exactly how many subs will be in the gear and which ones)
* Force set the values to values of our choice so we know what gear score to expect
* Run the get_gear_score() method and compare to our expected values

Results: ALL PASSED

### print_gear()

Function: Method to print the contents of the gear.

Source: 
* [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)

Test Modules:
* [test_print_gear.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_print_gear.py)
 
Test Plan Valid Entries:
* Create an empty Gear() object and test three different cases for rare, heroic, and epic gear
* Assign specific values for gear_grade, gear_type, gear_level, gear_set, mainstat_id, and some substat_id's
* Change mainstat and substat values to some arbitrary numbers, also test by switching is_reforged to True in one case.
* Write the expected output
* Use mock_print to compare the output to expected output.
    * One issue that arose with this is the formatting, trailing white spaces and \n commands were causing the tests to fail, even though the output and expected output looked the same visually, so we had to remove them from both case, standardize them, before comparing with self.assertEqual()

Results: ALL PASSED

### add_substat()

Function: Add a new substat to a gear that is not already in the gear. Cannot add more substats if gear already has 4 substats. Usual restrictions apply such as no duplicates, no conflict with main, gear_type, etc.

Source: 
* [gear.py](https://github.com/mesaqlain/e7_items/blob/main/src/gear.py)

Test Modules:
* [test_add_substat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_add_substat.py)
 
Test Valid Entries:
* Try add_substat() on rare gear twice, assert that the number of substats have increased, make sure ther are no duplicates, and ensure that the new stats are as expected.
* Try add_substat() on heroic gear once, assert that the number of substats have increased, make sure ther are no duplicates, and ensure that the new stats are as expected.

Test Invalid Entries:
* Cannot add_substat() on epic gear as it has 4 subs already.
* Cannot add_substat() on heroic gear more than once.
* Cannot add_substat() on rare gear more than twice.

Results: ALL PASSED